In [3]:
# =====================================================
# Data preparation for SpliceTransformer (ONE CELL)
# Just set OUTPUT_DIR and RUN
# =====================================================

import pandas as pd
from pyfaidx import Fasta
from Bio.Seq import Seq
import os
import time
from tqdm import tqdm

# ================= USER CONFIG (ONLY CHANGE THIS) =================
OUTPUT_DIR = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data"   # <-- CHỈ SỬA DÒNG NÀY
# =================================================================

# ================= FIXED CONFIG =================
GENOME_PATH = r"D:\Homo_sapiens.GRCh38.dna.primary_assembly.fa"
DATA_FOLDER = r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"

MODEL_MAX_LEN = 8192
CONTEXT = MODEL_MAX_LEN // 2   # 4096

FILES = ["test_1_1_1.csv", "test_2_1_1.csv", "test_4_1_1.csv", "test_10_1_1.csv", "test_data.csv"]
# ================================================


def get_sequence_worker(row, fasta_obj):
    """
    Extract centered sequence for SpliceTransformer.
    Output length = 8192, splice site exactly at center.
    """
    try:
        parts = row['id'].split('_')
        chrom, pos, strand = parts[1], int(parts[2]), parts[3]

        # Offset fix: keep splice dinucleotide centered
        actual_pos = pos - 1

        start = actual_pos - CONTEXT
        end = actual_pos + CONTEXT

        seq = str(fasta_obj[chrom][max(0, start):end]).upper()

        # Pad with N if touching chromosome boundary
        if start < 0:
            seq = ("N" * abs(start)) + seq
        if len(seq) < MODEL_MAX_LEN:
            seq = seq + ("N" * (MODEL_MAX_LEN - len(seq)))

        # Reverse complement if negative strand
        if strand == '-':
            seq = str(Seq(seq).reverse_complement())

        assert len(seq) == MODEL_MAX_LEN
        return seq

    except Exception:
        return "N" * MODEL_MAX_LEN


def diagnose_splice_sites(df, sample_size=5):
    """
    Quick sanity check: GT / AG must be at center.
    """
    center = MODEL_MAX_LEN // 2
    print(f"\n{'Type':<10} | {'Window (-2:+2)':<20} | Found?")
    print("-" * 55)

    for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
        subset = df[df['Splicing_types'] == label]
        if len(subset) == 0:
            continue

        samples = subset.sample(min(sample_size, len(subset)))
        for _, row in samples.iterrows():
            seq = row['sequence']
            window = seq[center-2:center+4]

            target = "GT" if label == 1 else "AG"
            display = window[:2] + "[" + window[2:4] + "]" + window[4:]
            found = "✅" if window[2:4] == target else "❌"

            print(f"{name:<10} | {display:<20} | {found}")


def run():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"[{time.strftime('%H:%M:%S')}] Loading genome...")
    genome = Fasta(GENOME_PATH, sequence_always_upper=True)

    for file_name in FILES:
        input_path = os.path.join(DATA_FOLDER, file_name)
        output_path = os.path.join(OUTPUT_DIR, file_name)

        if not os.path.exists(input_path):
            print(f"❌ Skip {file_name} (not found)")
            continue

        start_time = time.time()
        df = pd.read_csv(input_path)

        # Sort for faster disk access
        df[['_tmp_chr', '_tmp_pos']] = df['id'].str.split('_', expand=True)[[1, 2]]
        df['_tmp_pos'] = df['_tmp_pos'].astype(int)
        df = df.sort_values(['_tmp_chr', '_tmp_pos']).reset_index(drop=True)

        print(f"\n🚀 Processing {file_name} | {len(df)} rows")

        sequences = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Extracting"):
            sequences.append(get_sequence_worker(row, genome))

        df['sequence'] = sequences

        diagnose_splice_sites(df)

        df.drop(columns=['_tmp_chr', '_tmp_pos'], inplace=True)
        df.to_csv(output_path, index=False)

        elapsed = time.time() - start_time
        print(f"✅ Saved: {output_path}")
        print(f"⚡ Speed: {len(df)/elapsed:.2f} seq/s")

    print("\n🎯 DONE – Data ready for SpliceTransformer")


# ================= RUN =================
run()


[23:27:52] Loading genome...

🚀 Processing test_1_1_1.csv | 26310 rows


Extracting: 100%|██████████| 26310/26310 [00:01<00:00, 19478.59it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | GG[TG]AG             | ❌
Donor      | AT[GT]GA             | ✅
Donor      | GG[TA]AC             | ❌
Donor      | GG[TT]AG             | ❌
Donor      | GG[GT]AC             | ✅
Acceptor   | AG[AA]CC             | ❌
Acceptor   | AA[GG]AG             | ❌
Acceptor   | AG[GT]TG             | ❌
Acceptor   | AA[GT]GC             | ❌
Acceptor   | CA[GC]AA             | ❌
✅ Saved: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data\test_1_1_1.csv
⚡ Speed: 8509.29 seq/s

🚀 Processing test_2_1_1.csv | 35132 rows


Extracting: 100%|██████████| 35132/35132 [00:01<00:00, 26232.96it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | AG[GT]AT             | ✅
Donor      | AG[GT]AA             | ✅
Donor      | AA[GT]AA             | ✅
Donor      | AG[GT]AC             | ✅
Donor      | AT[GT]AA             | ✅
Acceptor   | AG[GT]GG             | ❌
Acceptor   | AG[CC]GG             | ❌
Acceptor   | CA[GG]TT             | ❌
Acceptor   | AG[GG]AC             | ❌
Acceptor   | TA[GA]TC             | ❌
✅ Saved: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data\test_2_1_1.csv
⚡ Speed: 9764.73 seq/s

🚀 Processing test_4_1_1.csv | 52776 rows


...

Extracting: 100%|██████████| 105708/105708 [00:04<00:00, 26223.29it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | CA[GT]GA             | ✅
Donor      | TG[GT]GA             | ✅
Donor      | GG[TA]AG             | ❌
Donor      | GG[GT]GG             | ✅
Donor      | GG[TC]TG             | ❌
Acceptor   | AG[AG]TG             | ✅
Acceptor   | AA[GG]AA             | ❌
Acceptor   | CA[GA]TC             | ❌
Acceptor   | CA[GA]CT             | ❌
Acceptor   | CA[GG]TG             | ❌
✅ Saved: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data\test_10_1_1.csv
⚡ Speed: 9681.08 seq/s

🚀 Processing test_data.csv | 938297 rows


Extracting: 100%|██████████| 938297/938297 [00:36<00:00, 26044.40it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | GG[TG]TG             | ❌
Donor      | AG[GT]AA             | ✅
Donor      | AC[GT]AA             | ✅
Donor      | AG[TT]CT             | ❌
Donor      | GG[TT]CT             | ❌
Acceptor   | TA[GG]GC             | ❌
Acceptor   | CA[GA]TC             | ❌
Acceptor   | TA[GG]GA             | ❌
Acceptor   | CA[GA]AA             | ❌
Acceptor   | AG[AC]AA             | ❌


KeyboardInterrupt: 